In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

#參數
lr = 0.001    #learning range
training_iters = 100000
batch_size = 128 #每次帶入的資料量（批量）（自設）
display_step = 10

n_inputs = 28   #圖像畫素 每行有28畫素
n_steps = 28    #圖像總共28行
n_hidden_units =64  #神經元數量（自設）
n_classes = 10    #預測0~9 分類十種結果


x = tf.placeholder(tf.float32, [None, n_steps, n_inputs])  #依照上述參數： 初始X_shape=[128, 28, 28]
y = tf.placeholder(tf.float32, [None, n_classes])            #依照上述參數： 初始y_shape=[128, 10]

weights = {'in': tf.Variable(tf.random_normal([n_inputs, n_hidden_units])),  #依照上述參數： 初始weights_in_shape=[28, 64]
          'out': tf.Variable(tf.random_normal([n_hidden_units, n_classes]))} #依照上述參數： 初始weights_out_shape=[64, 10]

biases = {'in': tf.Variable(tf.constant(0.1, shape=[n_hidden_units])), #依照上述參數： 初始biases_in_shape=[64]
         'out': tf.Variable(tf.constant(0.1, shape=[n_classes]))}     #依照上述參數： 初始biases_out_shape=[10]

#####################以下 重點區域 #########################

def RNN(x, weights, baises):
    # input layer
    ##############
    #為了 Wx+b 中的Wx(矩陣相乘) : W目前是二維[28, 64] x是三維[128, 28, 28] ，x要reshape
    x = tf.reshape(x, [-1, n_inputs])                   #x_shape=[128*28, 28] 二維
    x_in = tf.matmul(x, weights['in']) + biases['in']   #x_in_shape=[128*28, 64]
    #在reshape回三維
    x_in = tf.reshape(x_in, [-1, n_inputs, n_hidden_units])
    
    
    #cell
    #############
    
    lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden_units, forget_bias=1.0, state_is_tuple=True)
    init_state = lstm_cell.zero_state(batch_size, dtype=tf.float32)
    
    #lstm_cell會生成（c_state,h_state） c_state:主線  h_state:支線  
    #主線就是從init_state 不斷經過lstm_cell時被（三個gate：write複寫,forget修改,read讀取）影響改變（但沒有被tahn激勵）
    #而支線是根據新進資料激勵後 跟 被影響改變的主線且激勵後 揉合成一支線 用來輸出此階段（時間）結果
       
    cell_outputs, cell_final_states = tf.nn.dynamic_rnn(lstm_cell,x_in,  initial_state=init_state, time_major=False)
    
    #output layer
    #############
    results = tf.matmul(cell_final_states[1], weights['out']) + biases['out']   #計算當前階段結果，states用“支線”
    
    return results
    
#####################以上 重點區域 #########################
    
    
    
pred = RNN(x, weights, biases)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))

train_op = tf.train.AdamOptimizer(lr).minimize(cost)

correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))



with tf.Session() as sess:
    # tf.initialize_all_variables() no long valid from
    # 2017-03-02 if using tensorflow >= 0.12
    if int((tf.__version__).split('.')[1]) < 12:
        init = tf.initialize_all_variables()
    else:
        init = tf.global_variables_initializer()
    sess.run(init)
    step = 0
    while step * batch_size < training_iters:
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape([batch_size, n_steps, n_inputs])
        sess.run([train_op], feed_dict={
            x: batch_xs,
            y: batch_ys,
        })
        if step % 20 == 0:
            print(sess.run(accuracy, feed_dict={
            x: batch_xs,
            y: batch_ys,
        }))
        step += 1
 
    
    
    


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
0.09375
0.484375
0.5625
0.671875
0.773438
0.679688
0.859375
0.867188
0.84375
0.921875
0.804688
0.890625
0.9375
0.875
0.90625
0.921875
0.90625
0.90625
0.929688
0.898438
0.882812
0.953125
0.921875
0.953125
0.914062
0.9375
0.929688
0.953125
0.929688
0.9375
0.945312
0.96875
0.921875
0.929688
0.984375
0.953125
0.96875
0.929688
0.929688
0.945312
